In [1]:
import numpy as np
from scipy.stats import betaprime
from scipy.signal import find_peaks
from scipy.signal import argrelmin
from scipy.integrate import quad
from scipy.optimize import curve_fit
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Generador:
Clase que contiene los datos para generar señales con parámetros aleatorios.
Posee un método:
### generar_senial
Este método devuelve la señal simulada, con parámetros aleatorios.
Llamando a este método con argumentos adecuados, se puede que se cambien los parámetros del objeto
    

In [2]:
class Generador:
    '''
    Generador: Clase que encapsula los méetodos y xx necesarios para generar la simulación de medición.
    Al crear un objeto Generador se pueden ingresar los argumentos:
        -picos: Lista de los tiempos donde van a estar centrados los pulsos
        -prob: Lista de las probabilidades de ocurrencia de los picos
        -muestras: Entero que indica cuántos puntos se generan entre t=0 y t=10 microsegundos

    Posee solo dos métodos:
        -generar_senial(), encargada de simular una señal entre 0 y 10 microsegundos, contemplando el lugar de los picos y 
         la probabilidad de ocurrencia de cada uno, además de distintas señales de ruido.
         En la simulación, la posición real de los picos varía con una distribución Beta prima, con parámetros a=2 y b=4.
         Devuelve dos arrays de numpy, el primero contiene los valores de tiempo (eje X) y el segundo los valores de la señal (eje Y)
        
        -muestrear(), se encarga de realizar 'nmed' veces la simulación de la señal, utilizando la función generar_senial().
         Devuelve dos arrays de numpy, el primero contiene los valores de tiempo (eje X) y el segundo los valores promedios los valores simulados

    
    '''
    def __init__(self,picos:list=[1.8, 2.8, 5],prob:list=[0.3, 0.1, 0.6], muestras:int=1000):
        self.picos=picos                        #Valores de tiempo donde se centran los picos
        self.P=prob                             #Probabilidades de ocurrencia de los picos
        self.muestras=muestras
        self.t=np.linspace(0, 10, muestras)     #define los puntos de muestreo
        

    def generar_senial(self, **kwargs):
        ''' Función que genera una señal en milivoltios, barrida entre 0 y 10 micro segundos.
        Devuelve dos arrays de numpy, el primero contiene los valores de tiempo (eje X) y el segundo los valores de la señal (eje Y)
        Al llamarla, se pueden ingresar los siguientes kwargs:
         -picos     -> Lista con los tiempos donde se centran los picos
         -prob      -> Lista de probabilidad de ocurrencia que tiene cada pico 
         -muestras  -> Número de muestras en las que se divide el tiempo (0 - 10 uS) 

         Si estos kwargs no se brindan, se toman los parámetros definidos en el objeto.
         Si se brindan como argumento, modifican a los definidos en el objeto
         '''
        
        #Si se declaran los picos o las probabilidades, se cambian en la raiz
        if 'picos' in kwargs: self.picos=kwargs['picos']
        if 'prob' in kwargs: self.P=kwargs['prob'] 
        if 'muestras' in kwargs:
            self.muestras=kwargs['muestras']
            self.t=np.linspace(0, 10, kwargs['muestras'])
            
        ####################################################################
        ##### A partir de acá se generan las señales base de los picos #####
        ####################################################################

        a,b,sigma_t = 4,2,0.15                  #Seteo los parámetros para la distribución beta prima

        taux=[]                                 #Determmino las posiciones reales de los picos siguiendo una
        for tx in self.picos:                   #distribución beta prima
            t = betaprime.rvs(a, b, loc=tx, scale=sigma_t, size=1)
            taux.append(t[0])
        t0s=np.array(taux)
        
        sigmas=np.array([np.random.normal(0.05, 0.005) for x in self.picos])      #Calculo los anchos de los pulsos(sigmas)    
        omegas=np.array([np.random.normal(5*np.pi, 0.05) for x in self.picos])    #Calculo las frecuencias de oscilacion

        #Probabilidades de los picos
        rx=np.random.random(size=len(self.picos))             #obtengo tres valores random entre 0 y 1
        px=np.array(self.P)                                   #Convierto probabilidades a NP
        aciertos=np.array([r<=p for r,p in zip(rx,px)])       #En los aciertos se establece TRUE si el pico aparece y FALSE si no
    
        #Determino los valres de A
        A = 1 / (sigmas*np.sqrt(2 * np.pi))
        #pulsos = A * np.exp(-(self.t - t0s) ** 2 / (2 * sigmas ** 2)) * np.cos(omegas * (t - t0s + sigmas))
        pulsosparciales =np.array([match*Ax * np.exp(-((self.t - t0) ** 2 )/ ((2 * sigma ** 2))) * np.cos(omega * (self.t - t0 + sigma)) for
                                          Ax, t0,sigma,omega,match in zip(A,t0s,sigmas,omegas,aciertos)])
        pulsos=pulsosparciales.sum(axis=0)
       

        ####################################################################
        ### A partir de acá se generan las señales "accesorio" (ruidos)  ###
        ####################################################################

        #obtengo el ruido absoluto
        ruido_abs = np.random.normal(0, 0.2, size=self.muestras)
        #Obtengo el ruido relativo
        ruido_rel = np.random.normal(1, 0.1, size=self.muestras)
        #Obtengo la señal de fondo
        fondo=(2-self.t) / 100

        ####################################################################
        #### Sumatoria de todas las señales para formar la señal final  ####
        ####################################################################

        pulso=pulsos*ruido_rel+ruido_abs+fondo

        return self.t, pulso
    
    def muestrear(self, picos:list, prob:list ,nmed=10000):
        '''
        Método que promedia de un número 'nmed' de señales generadas con el método 'generar_senial()'
        Devuelve dos arrays de numpy, el primero contiene los valores de tiempo (eje X) y el segundo los valores promedios los valores simulados

        Al llamarla, se deben ingresar:
         -picos     -> Lista con los tiempos donde se centran los picos (obligatorio)
         -prob      -> Lista de probabilidad de ocurrencia que tiene cada pico (obligatorio)     
         -muestras  -> Número de muestras en las que se divide el tiempo (0 - 10 uS)  (Opcional, por defecto 10000)
        
        Ejemplo:
            tiempos, valores=Generador.muestrear(nmed=500, picos=[1.8,2.8,5], prob=[0.6,0.1,0.3])
            -> tiempos= nparray con los puntos de tiempos tomados entre 0 y 10 uS
            -> valores= nparray con los valores promedio de 500 muestras, correspondientes a esos tiempos

        De manera opcional, se podría ver el set de datos generados (las 'nmed' mediciones con todos sus valores) accediendo al atributo 'dataset'
        '''
        dataset=np.empty((nmed,self.muestras))
        for x in range(nmed):
            senial=self.generar_senial(picos=picos, prob=prob)[1]
            dataset[x]=senial
        self.dataset= dataset

        return self.t, dataset.sum(axis=0)/nmed

Creo el objeto que genera las señales.
$\textbf{'muestras'}$ indica el muestreo de tiempo entre 0  y 10 $\mu$ Segundos (en este caso 5000), es decir, en cuantos puntos se divide

In [3]:
sim=Generador(muestras=1000)

Animo el objeto para generar el 'display'

In [4]:
# Animación
#Genero y configuro la gráfica
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_xlim(0, 10)
ax.set_ylim(-5, 15)
ax.set_xlabel('Tiempo [μS]')
ax.set_ylabel('Tensión [mV]')
ax.grid(linestyle=':',which='major',alpha=0.5, color='gray')
fig.suptitle('Simulación del display de medición', fontsize=14)



line, = plt.plot([], [])

#En cada frame de la animación se muestra un conjunto de datos que representa la señal en ese instante.
#Para eso se utiliza el método 'generar_senial' de la clase 'Generador'
def animate(i,L):
    t,sig=sim.generar_senial()
    L.set_data(t, sig)
    return L,

#Realizo la animación con 100 muestras, luego se repite
repeticiones=100
animacion=animation.FuncAnimation(fig, animate,repeticiones,fargs=(line,), interval=250, blit=True)
plt.show()

#### Datasets
A continuación se generan los sets de datos para tratar

In [5]:
#creo x e y, ambas son una lista donde en cada miembro será un array de np con las n mediciones en x e y respectivamente
x=[0,0,0]
y=[0,0,0]

#Determino los tres pares de conjuntos (x e y), cada uno con distintas posiciones de picos y probabilidades
#Para esto utilizo el método 'muestrear' de la clase 'Generador'
x[0],y[0]=sim.muestrear(nmed=10000, picos=[1.8,2.8,5], prob=[1,0,0])
x[1],y[1]=sim.muestrear(nmed=10000, picos=[1.8,2.8,5], prob=[0.8,0.1,0.1])
x[2],y[2]=sim.muestrear(nmed=10000, picos=[1.8,2.8,3.5], prob=[0.85,0.1,0.05])

#### Puntos clave
Encuentro los puntos clave: Máximos, puntos de altura mitad y mínimos de los picos

In [6]:
#Genero listas para los máximos, valores de altura mitad (medizq y medder), y los mímimos o valles de los picos
maximos=[0,0,0]
medizq=[0,0,0]
medder=[0,0,0]
minimos=[0,0,0]  #Cáda miembro tendrá una lista. Cada lista tendrá los máximos y mínimos para cada pico 

#Encuentro los picos haciendo uso de la función 'find_peaks' de scipy.signal
#Con el argumento 'prominence' hago un filtrado de los picos con prominencia suficiente para que sea real
#La función devuelve la posición de los picos, así como las alturas medias izquierda y derecha (entre otras cosas)
for n, (xn ,yn) in enumerate(zip(x,y)):     #Hago un bucle para los tres datasets
    maximos[n], prop = find_peaks(yn,prominence=0.02, width=10)
    medizq[n]=prop["left_ips"]
    medder[n]=prop["right_ips"]
    #Obtengo los posibles valles de los datosm usando la funcion 'argrelmin' de scipy.signal
    #order indica cuántos puntos alrededor debe buscar para considerarse mínimo
    mnm,=argrelmin(yn, axis=0, order=10, mode='clip')
    minimos[n]=[]
    #Analizo para cada pico, cuales son los valles mas cercanos y almaceno los puntos en forma de lista.
    #Para esto hago un filtrado de los mínimos devueltos por argrelmin, 
    # y luego me quedo con el ultimo de los menores y el primero de los mayores
    #A cada pico de cada conjunto de muestras le corresponde una lista con dos valores de mínimos (derecha e izq)
    for mx in maximos[n]:
        ix=mnm[mnm<mx][-1]
        dx=mnm[mnm>mx][0]
        minimos[n].append([ix,dx])
        

## Fiteo de las gráficas, integral por cuadrados mínimos
Para obtener el área bajo las curvas, es necesario ajustar (fitear) los datos a una función conocida
Procedo recortando cada gráfica por los puntos considerados mínimos (array minimos), y aplico el método $\textbf{curve fit()}$ 
de $\textbf{'scipy.optimize'}$ en cada recorte. Teniendo esto en cuenta, cada set de datos tendrá un grupo de parámetros por cada pico que tenga

In [7]:
#Defino la función
def fiteo(t, a1, m1, s1,sa1, sm1, ss1):
    #Dos gaussianas por cada pico:
        #Quise hacer que cada pico sea una suma de dos gaussianas, pero en algunas simulaciones el fiteo daba error,
        #sobre todo en el tercer caso con picos [1.8,2.8,3.5], el pico mas pequeño es el que más error genera
        #Por esto decido hacer un fiteo menos presiso, tomando solo una gaussiana, y que funcione en todos los casos
    
    r1a=a1* np.exp(-(((t - m1)**2 )/ (2*(s1**2))))
    r1b=sa1* np.exp(-(((t - sm1)**2 )/ (2*(ss1**2))))

    return r1a# +r1b

Para ayudar a la funcion curve_fit, se deben ingresar parámetros iniciales de la función (estimados).
######
Teniendo la curva de ajuste:

$f(x) = Ae^{-\frac{(x-\mu)^2}{2\sigma^2}}$

Existe una relación entre los parámetros de la función y los datos brindados para generarlos:

-La altura A es proporcional a las probabilidades de ocurrencia de los picos.

-La media $\mu$ es cercana al punto donde se definieron los picos

Teniendo en cuenta esto, los parámetros estimados para la funcion se pueden establecer con esos valores:

In [8]:
#Con los siguientes datos genero los parámetros estimados (p0) para cada fiteo
#Las listas cuyo nombre empieza con 'sub', solo son válidas para el modelo de fiteo con dos gaussianas,
#representando valores para la gaussiana menos prominente
picos=[[1.8,2.8,5],[1.8,2.8,5],[1.8,2.8,3.5]]
subpicos=[[2,1,3,5.2],[2.1,3,5.2],[2,3,3.6]]  
altura=[[0.9,0,0],[0.8,0.05,0.05],[0.8,0.1,0.1]]
subaltura=[[0.4,0.02,0.02],[0.3,0.1,0.05],[0.2,0.05,0.05]] #->Para el modelo de fiteo con dos gaussianas, representa el segundo pico con menor altura
desv=[[0.1,0.15,0.15],[0.1,0.05,0.05],[0.15,0.15,0.15]]
subdesv=[[0.2,0.3,0.3],[0.2,0.2,0.2],[0.3,0.3,0.3]]

#En esta lista se almacenarán los parámetros calculados con curve_fit.
popt=[[],[],[]]
#En esta lista se almacenarán las áreas calculadas .
areas=[[],[],[]]

for figura,mxx in enumerate(maximos):
    #Para cada set de datos 
    
    for posicion,valor in enumerate(mxx):
        #para cada pico encontrado
        #calculo los parámetros iniciales para el fiteo
        p0=[altura[figura][posicion],picos[figura][posicion],desv[figura][posicion],
        subaltura[figura][posicion],subpicos[figura][posicion],subdesv[figura][posicion]]
        #Delimito los datos para el fiteo a la anchura del pico
        rangopico=minimos[figura][posicion]
        x_=x[figura][rangopico[0]:rangopico[1]]
        y_=y[figura][rangopico[0]:rangopico[1]]
        #Obtengo los parámetros del fiteo
    
        poptx,cov = curve_fit(fiteo, x_, y_, p0)
        
        popt[figura].append(poptx)

        #Calculo area bajo la curva. Para esto uso el método quad de scipy
        areax,_=quad(fiteo,10*rangopico[0]/sim.muestras,10*rangopico[1]/sim.muestras,args=tuple(poptx))
        areas[figura].append(areax)   


c:\Users\Juan\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


#### Gráficos
Se generan tres gráficos, uno por dataset. Cada gráfico tiene dos subgráficos: El superior repesenta la curva promedio con todos sus puntos. El inferior representa el fiteo

En cada subgráfico se muestran los picos, los valores de anho a altura mitad, están marcados los 'valles' o 'mínimos'

Además, en las curvas de fiteo se colorean las áreas integradas, y se muestra el valor de la integración

In [11]:
#Defino los títulos
titulos=["Picos en t=1.8,2.8,5 y probabilidades 1,0,0",
"Picos en t=1.8,2.8,5 y probabilidades 0.8,0.1,0.1", "Picos en t=1.8,2.8,3.5 y probabilidades 0.85,0.1,0.05"]

#Hago un bucle para generar todos los gráficos
for rep,(xn,yn,mxn,mnm,titx) in enumerate(zip (x,y,maximos,minimos, titulos)):
    
    #Creo la figura y los ejes para graficar
    fig, axs = plt.subplots(2, 1,figsize=(8, 8))
    fig.suptitle(f'Gráficos de set de datos {rep+1}', fontsize=14)
   
    #Configuro las grillas
    axs[0].grid(linestyle=':',which='minor',alpha=0.2, color='gray')
    axs[0].grid(linestyle=':',which='major',alpha=0.6, color='black')
    axs[1].grid(linestyle=':',which='major',alpha=0.6, color='black')

    #Grafico los datos promedio
    axs[0].plot(xn,yn, label='Promedios')
    
    #Grafico los puntos máximos
    #El valor en x obtenido de los máximos corresponde a una posicion en toda la muestra.
    #Por eso es necesario adaptarlos multiplicando por 10 (t=0 a t=10 uS) y dividirlo
    #por la cantidad de muestras de la simulación 
    axs[0].plot(mxn*10/sim.muestras,yn[mxn],'r^',markersize=4, label='Máximos')
    
    #Grafico los puntos 'valle'
    vallen=[]
    for vn in mnm:
        for valor in vn:
            vallen.append(valor)
    
    valles=np.array(vallen)
    axs[0].plot(valles*10/sim.muestras,yn[valles],'mv',markersize=4,label='Mínimos')
    
    #Grafico los valores mitad
    medios=(np.rint(np.append(medizq[rep],medder[rep]))).astype(int)
    axs[0].plot(medios*10/sim.muestras,yn[medios],'ko',markersize=3,label='Medios' )
    axs[0].plot(medios*10/sim.muestras,yn[medios],'wo',markersize=1 )

    #Escribo los textos de máximos y de anchos en el gráfico
    for n,m in enumerate(mxn):
        text="max {}: {:.3}".format(n+1,yn[m])
        axs[0].annotate(text,xy= (m*10/sim.muestras-0.2,yn[m]+0.05))

    #Para determinar el ancho a altura mitad, hago la diferencia entre 
    #los valores arrojados por el método 'find_peaks' de scipy.signal
    for n,(i,d) in enumerate(zip(medizq[rep],medder[rep])):
        ancho=(d-i)*10/sim.muestras
        text="ancho {}: {:.3} $\mu$S".format(n+1,ancho)
        axs[0].annotate(text,xy= (5,1.2-0.15*n))

    #Grafico los fiteos
    for posicion,popx in enumerate(popt[rep]):
        rangopico=minimos[rep][posicion]
        if posicion==0:
            y_=fiteo(xn,*popx)      #Calculo los valores de y para cada pico
        else:
            y_=y_+fiteo(xn,*popx)
    y_=y_+(2-xn)/100                #Como sé la pendiente de las señales, la agrego
    axs[1].plot(xn,y_, 'r:',linewidth=2,label='fiteos')
    
    #Aplico relleno en los fiteos
    for lim in minimos[rep]:
        x_=x[rep][lim[0]:lim[1]]
        axs[1].fill_between(x_,(2-x_)/100,y_[lim[0]:lim[1]], alpha=0.5)
    
    #Escribo los textos de cada área en el gráfico de fiteo
    for n,(area,mx) in enumerate(zip(areas[rep],mxn)):
        text="Área {}: {:.4}".format(n+1,area)
        axs[1].annotate(text,xy= (mx*10/sim.muestras-0.2,yn[mx]))
      

    
    #títulos y labels
    axs[0].set_title(titx)
    axs[0].set_xlabel('Tiempo ($\mu$ segundos)')
    axs[0].set_ylabel('Tensión (mV)')
    axs[1].set_xlabel('Tiempo ($\mu$ segundos)')
    axs[1].set_ylabel('Tensión (mV)')

    #Límites de ejes
    axs[0].set_ylim([-0.2,1.4])
    axs[1].set_ylim([-0.2,1.4])

       
    #Otros accesorios
    axs[0].minorticks_on()
    axs[0].legend(loc="best", title="Referencias")
    axs[1].legend(loc="best", title="Referencias")
    
plt.show()